AKA T1

In [5]:
import re

In [49]:
def remove_comments(text):
    # Split the text into lines
    lines = text.split('\n')
    # Filter out lines that don't start with "%"
    filtered_lines = [line for line in lines if not line.strip().startswith('%')]
    # Join the filtered lines back together
    cleaned_text = '\n'.join(filtered_lines)
    return cleaned_text

In [57]:
def remove_simples(text):
    text = text.replace("$$", "$")
    text = text.replace("\\begin{equation}", "$")
    text = text.replace("\\end{equation}", "$")
    text = text.replace("\\begin{equation*}", "$")
    text = text.replace("\\end{equation*}", "$")
    text = text.replace("\\[", "$")
    text = text.replace("\\]", "$")
    text = text.replace("\\(", "$")
    text = text.replace("\\)", "$")
    text = text.replace(".$", "$.")
    text = text.replace("$", " $ ")
    text = text.replace("-", " - ")
    text = text.replace("%", "")
    text = text.replace("\\item", "")
    return text

In [96]:
def remove_figures_and_exercises(text):
    pattern = r"\\begin\{figure\}(.*?)\\end\{figure\}"
    # Use the sub() function to remove all matches of the pattern
    new_text = re.sub(pattern, "", text, flags=re.DOTALL)
    pattern2 = r"\\begin\{exercises\}[^\}]+\\end\{exercises\}"
    # Use the sub() function to remove all matches of the pattern
    new_text = re.sub(pattern2, "", new_text,flags=re.DOTALL)
    return new_text

In [89]:
def remove_figure_content(text):
    # Define the regular expression pattern
    pattern = r'\\begin\{figure\}.*?\\end\{figure\}'
    
    # Replace all occurrences of the pattern with an empty string
    clean_text = re.sub(pattern, '', text, flags=re.DOTALL)
    
    return clean_text

# Example usage:
text = """
Some text before the figure.
\begin{figure}
    Figure content to be removed.
\end{figure}
Some text after the figure.
"""

cleaned_text = remove_figure_content(text)
print(cleaned_text)


Some text before the figure.
egin{figure}
    Figure content to be removed.
\end{figure}
Some text after the figure.



In [8]:
def remove_environments(text):
    
    pattern = r'\\begin\{[^\}]+\}'
    # Substitute the pattern with an empty string
    cleaned_text = re.sub(pattern, '', text)
    pattern2 = r'\\end\{[^\}]+\}'
    # Substitute the pattern with an empty string
    cleaned_text = re.sub(pattern2, '', cleaned_text)
    return cleaned_text

In [9]:
def remove_emphasis(text):
    text = re.sub(r"\\demph\{([^{}]*)\}", r"\1", text)
    text = re.sub(r"\\emph\{([^{}]*)\}", r"\1", text)
    return text

In [58]:
def remove_formatting(text):
    text = re.sub(r'~\\[^}]+}', ' $ X $ ', text)
    text = re.sub(r'\\nearby[^\}]+\}', ' $ X $ ', text)

    text = re.sub(r"\\chapter\{.*?\}", "", text)
    text = re.sub(r"\\section\{.*?\}", "", text)
    text = re.sub(r"\\subsection\{.*?\}", "", text)
    text = re.sub(r"\\label\{.*?\}", "", text)
    text = re.sub(r"\\ntn\{.*?\}", "", text)
    text = re.sub(r"\\index\{.*?\}", "", text)
    text = re.sub(r"\\ref\{.*?\}", "", text)
    text = re.sub(r"\\bref\{.*?\}", "", text)
    text = re.sub(r"\\eqref\{.*?\}", "", text)

    return text

In [11]:
def prep_w_spaces(text):
    text = text.replace("\\\\", " \\")
    text = text.replace("\{", "{")
    text = text.replace("\}", " } ") 
    text = text.replace(",", ", ")
    return text

In [12]:
def list_fix(text):
    text = text.replace("1.", "(i)")
    text = text.replace("2.", "(ii)")
    text = text.replace("3.", "(iii)")
    text = text.replace("4.", "(iv)")
    text = text.replace("5.", "(v)")
    text = text.replace("6.", "(vi)")
    text = text.replace("7.", "(vii)")
    text = text.replace("8.", "(viii)")
    text = text.replace("9.", "(ix)")
    text = text.replace("10.", "(x)")

    return text

In [13]:
def final_sweep(text):
    words = [word.replace("\\", "") for word in text.split() if not word.startswith("Helvetica")]
    text = " ".join(words)
    return text

In [59]:
def clean_everything(text):
    text = remove_comments(text)
    text = remove_simples(text)
    text = remove_figures_and_exercises(text)
    text = remove_environments(text)
    text = remove_emphasis(text)
    text = remove_formatting(text)
    text = prep_w_spaces(text)
    text = list_fix(text)
    text = final_sweep(text)
    return text

In [98]:
import spacy

from spacy_conll import init_parser

nlp = init_parser("en_core_web_sm", "spacy")
from spacy.language import Language
@Language.component("detextor")
def detextor(doc):
    dollar_indices = [index for index, token in enumerate(doc) if token.text == "$"]
    while len(dollar_indices) > 1:
        with doc.retokenize() as retokenizer:
            retokenizer.merge(doc[dollar_indices[0]:dollar_indices[1] + 1])
        dollar_indices = [index for index, token in enumerate(doc) if token.text == "$"]
    return doc
# nlp.remove_pipe("detextor") 
# you might need to add the above line back in if you run this block more than once
nlp.add_pipe("detextor", after="tagger")           

<function __main__.detextor(doc)>

In [99]:
adj = open("/Users/lucyhorowitz/Documents/GitHub/definition-extraction/textbooks/bct/adj.tex","r").read()
arl = open("/Users/lucyhorowitz/Documents/GitHub/definition-extraction/textbooks/bct/arl.tex","r").read()
cfnt = open("/Users/lucyhorowitz/Documents/GitHub/definition-extraction/textbooks/bct/cfnt.tex","r").read()
gaft = open("/Users/lucyhorowitz/Documents/GitHub/definition-extraction/textbooks/bct/gaft.tex","r").read()
intro = open("/Users/lucyhorowitz/Documents/GitHub/definition-extraction/textbooks/bct/intro.tex","r").read()
lims = open("/Users/lucyhorowitz/Documents/GitHub/definition-extraction/textbooks/bct/lims.tex","r").read()
rep = open("/Users/lucyhorowitz/Documents/GitHub/definition-extraction/textbooks/bct/rep.tex","r").read()
sets = open("/Users/lucyhorowitz/Documents/GitHub/definition-extraction/textbooks/bct/sets.tex","r").read()

tex = [adj, arl, cfnt, gaft, intro, lims, rep, sets]

with open("textbooks/bct/all_clean.txt","a") as a:
    for thing in tex:
        a.write(clean_everything(thing))


In [100]:
text = open("/Users/lucyhorowitz/Documents/GitHub/definition-extraction/textbooks/bct/all_clean.txt","r").read()
sent_lengths = []
with open("textbooks/bct/bct.conllu", "a") as f:
        text = clean_everything(text)
        doc = nlp(text)
        j = 1
        for sent in doc.sents:
            doc2 = nlp(sent.text)
            conll = doc2._.conll_str
            sent_lengths.append(len(doc2))
            f.write("# sent_id = " + str(j) + "\n")
            f.write("# text = " + sent.text + "\n")
            f.write(conll + "\n")
            j = j + 1

In [102]:
sents = open("textbooks/bct/bct_sents.txt","a")
with open("textbooks/bct/bct.conllu","r") as a:
    for line in a.readlines():
        if "# text = " in line:
            sents.write(line[9:])